# Identify the correct households based on the metadata and the specified inputs

1) Get metadata file
2) Read "input" values and select appropriate ones for each scenario
3) Identify building IDs associated with those properties
4) Save building IDs as CSV to be queried and aggregated in Step 1

For Scenario (1), with "no major electrification upgrades":

a) stove = gas
b) heat = gas
c) water heat = gas
d) solar = no
e) no AC
f) has occupancy

Each of these correspond to NREL ResStock inputs.

a) in.cooking_range = "Gas, 120% Usage" or "Gas, 100% Usage", or "Gas, 80% Usage" not "Electric, ..." or "Propane, ..."
b) in.heating_fuel = "Natural Gas" not "Electricity"
c) in.water_heater_fuel = "Natural Gas" not "Electricity"
d) in.has_pv = "No" not "Yes"
e) in.hvac_cooling_type = "" not "Central AC" or "Room AC"
f) in.vacancy_status = "Occupied" not "Vacant"

In [4]:
# Read the metadata file
import os
import pandas as pd

scenario = "scenario-baseline"
housing_type = "single-family-detached"

metadata = pd.read_csv(f"alameda/{scenario}/step0_in_CA_baseline_metadata_and_annual_results.csv")

metadata.head()


/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_1120/1079477664.py:8: DtypeWarning: Columns (124) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(f"alameda/{scenario}/step0_in_CA_baseline_metadata_and_annual_results.csv")


,bldg_id,upgrade,weight,applicability,in.sqft,in.representative_income,in.ahs_region,in.aiannh_area,in.area_median_income,in.ashrae_iecc_climate_zone_2004,...,out.emissions.propane.lrmer_high_re_cost_15.co2e_kg,out.emissions.propane.lrmer_low_re_cost_15.co2e_kg,out.emissions.propane.lrmer_mid_case_15.co2e_kg,out.emissions.propane.lrmer_mid_case_25.co2e_kg,out.bills.all_fuels.usd,out.bills.electricity.usd,out.bills.fuel_oil.usd,out.bills.natural_gas.usd,out.bills.propane.usd,out.energy_burden.percentage
0,5,0,252.301639,True,2179,74909.0,"CBSA Riverside-San Bernardino-Ontario, CA",No,100-120%,3B,...,0.0,0.0,0.0,0.0,3699.088937,2929.338937,0.0,769.75,0.0,4.94
1,12,0,252.301639,True,2678,74877.0,"CBSA Riverside-San Bernardino-Ontario, CA",No,120-150%,3B,...,0.0,0.0,0.0,0.0,4946.333651,4723.233651,0.0,223.10,0.0,6.61
2,31,0,252.301639,True,1228,85012.0,"CBSA Riverside-San Bernardino-Ontario, CA",No,100-120%,3B,...,0.0,0.0,0.0,0.0,2497.647905,2211.727905,0.0,285.92,0.0,2.94
3,38,0,252.301639,True,623,42184.0,"CBSA Los Angeles-Long Beach-Anaheim, CA",No,30-60%,3B,...,0.0,0.0,0.0,0.0,1061.153431,793.313431,0.0,267.84,0.0,2.52
4,55,0,252.301639,True,1698,150186.0,"CBSA Los Angeles-Long Beach-Anaheim, CA",No,150%+,3B,...,0.0,0.0,0.0,0.0,2476.945965,1966.655965,0.0,510.29,0.0,1.65


In [9]:
metadata_alameda = metadata[metadata["in.county"] == "G0600010"]

print(metadata_alameda.count())
metadata_alameda.head()

metadata = metadata_alameda

bldg_id                         2443
upgrade                         2443
weight                          2443
applicability                   2443
in.sqft                         2443
                                ... 
out.bills.electricity.usd       2443
out.bills.fuel_oil.usd          2443
out.bills.natural_gas.usd       2443
out.bills.propane.usd           2443
out.energy_burden.percentage    2443
Length: 289, dtype: int64


In [10]:
metadata["in.hvac_cooling_type"].unique()

array(['Room AC', nan, 'Ducted Heat Pump', 'Central AC',
       'Non-Ducted Heat Pump'], dtype=object)

In [11]:
# Desired filter conditions
cooking_range_condition = metadata["in.cooking_range"].isin(["Gas"])
heating_fuel_condition = metadata["in.heating_fuel"] == "Natural Gas"
water_heater_fuel_condition = metadata["in.water_heater_fuel"] == "Natural Gas"
solar_condition = metadata["in.has_pv"] == "No"
hvac_cooling_type_condition = metadata["in.hvac_cooling_type"].isna()
occupancy_condition = metadata["in.vacancy_status"] == "Occupied"
housing_type_condition = metadata["in.geometry_building_type_recs"] == "Single-Family Detached"

# Combine conditions
filtered_metadata = metadata[
    cooking_range_condition & # OK = 1493
    heating_fuel_condition & # OK = 1761
    water_heater_fuel_condition & # OK = 1934
    solar_condition & # OK = 2359
    hvac_cooling_type_condition & # OK = 721
    occupancy_condition & # OK = 2317
    housing_type_condition
]
# When all filters are applied, we get max 721 results... in this case, 305 buildings
# Therefore there are 305 buildings that satisfy our conditions

print(filtered_metadata.count()[0] < metadata.count()[0])
print("****" * 10)
print(filtered_metadata.count())


filtered_metadata.head()

True
****************************************
bldg_id                         178
upgrade                         178
weight                          178
applicability                   178
in.sqft                         178
                               ... 
out.bills.electricity.usd       178
out.bills.fuel_oil.usd          178
out.bills.natural_gas.usd       178
out.bills.propane.usd           178
out.energy_burden.percentage    178
Length: 289, dtype: int64


/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_1120/3866793590.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(filtered_metadata.count()[0] < metadata.count()[0])
/var/folders/gr/g6k4s8t13n98c8bw4ywdpxrr0000gn/T/ipykernel_1120/3866793590.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(filtered_metadata.count()[0] < metadata.count()[0])


,bldg_id,upgrade,weight,applicability,in.sqft,in.representative_income,in.ahs_region,in.aiannh_area,in.area_median_income,in.ashrae_iecc_climate_zone_2004,...,out.emissions.propane.lrmer_high_re_cost_15.co2e_kg,out.emissions.propane.lrmer_low_re_cost_15.co2e_kg,out.emissions.propane.lrmer_mid_case_15.co2e_kg,out.emissions.propane.lrmer_mid_case_25.co2e_kg,out.bills.all_fuels.usd,out.bills.electricity.usd,out.bills.fuel_oil.usd,out.bills.natural_gas.usd,out.bills.propane.usd,out.energy_burden.percentage
297,2917,0,252.301639,True,1698,63747.0,"CBSA San Francisco-Oakland-Hayward, CA",No,60-80%,3C,...,0.0,0.0,0.0,0.0,2223.466122,1018.256122,0.0,1205.21,0.0,3.49
392,3783,0,252.301639,True,2179,189249.0,"CBSA San Francisco-Oakland-Hayward, CA",No,150%+,3C,...,0.0,0.0,0.0,0.0,1475.197903,1013.007903,0.0,462.19,0.0,0.78
415,3978,0,252.301639,True,3310,280058.0,"CBSA San Francisco-Oakland-Hayward, CA",No,150%+,3C,...,0.0,0.0,0.0,0.0,2710.802871,1611.902871,0.0,1098.90,0.0,0.97
1251,12375,0,252.301639,True,1698,270137.0,"CBSA San Francisco-Oakland-Hayward, CA",No,150%+,3C,...,0.0,0.0,0.0,0.0,2456.930387,1710.290387,0.0,746.64,0.0,0.91
1296,12817,0,252.301639,True,1228,89096.0,"CBSA San Francisco-Oakland-Hayward, CA",No,80-100%,3C,...,0.0,0.0,0.0,0.0,2923.217767,2096.067767,0.0,827.15,0.0,3.28


In [14]:
filtered_metadata[0:1]["in.geometry_building_type_recs"]

297    Single-Family Detached
Name: in.geometry_building_type_recs, dtype: object

In [16]:
# What do we want to call this scenario? What do these properties correspond to?
scenario = "scenario-baseline"
housing_type = "single-family-detached"

# Save the 305 building ids to a CSV
filtered_metadata.to_csv(f"alameda/{scenario}/{housing_type}/step0_out_suitable_buildings.csv", index=False)